# Домашнее задание 1

$\textbf{Примечание:}$ запускайте ячейки только "сверху вниз" потому что в разных задачах одни и те же переменные (которые мы получаем из файлов с данными) имеют разные значения

In [1]:
import cvxpy as cp
import numpy as np
import math

![7.15](opt_7_15.jpg)

## Решение

$\textbf{var}(a_i^TZ) = a_i^T\textbf{var}(Z)a_i = a_i^T\Sigma a_i$ - линейная по $\Sigma$ функция.\
Нам нужно решить следующую задачу:
$$max_{i\neq j}|\rho_{ij}| \rightarrow min $$
$$ a_i^T\Sigma a_i = \sigma_i^2, \; i=1,...,m$$
$$ \Sigma \succeq 0 $$
При этом функция $|\rho_{ij}| = \frac{|\Sigma_{ij}|}{\sqrt{\Sigma_{ii}\Sigma_{jj}}}$ является квазивыпуклой (числитель - неотрицательная выпуклая функция, знаменатель - положительная вогнутая функция). Значит, и максимум от этой функции будет квазивыпуклым. Таким образом, мы можем найти минимум $\rho^{max}$, решая следуюущую задачу:
$$t \rightarrow min $$
$$ |\Sigma_{ij}| \leq t\sqrt{\Sigma_{ii}\Sigma_{jj}} $$
$$ a_i^T\Sigma a_i = \sigma_i^2, \; i=1,...,m $$
$$ \Sigma \succeq 0 $$
Оптимальное значение для $t$ и будет решением нашей задачи

In [2]:
from correlation_bounds_data import *

In [4]:
Sigma = cp.Variable((n,n), symmetric=True)
t = cp.Parameter(nonneg=True)
rho_cons = []

for i in range(n - 1):
    for j in range(i + 1, n):
        Sij = cp.vstack([Sigma[i, i], Sigma[j, j]])
        rho_cons += [cp.abs(Sigma[i, j]) <= t * cp.geo_mean(Sij)]

var_cons = [A[:, i].T @ Sigma @ A[:, i] == sigma[i]**2 for i in range(m)]
problem = cp.Problem(cp.Minimize(0), rho_cons + var_cons)
lb, ub = 0.0, 1.0
Sigma_opt = None

while ub - lb > 1e-3:
    t.value = (lb + ub) / 2
    problem.solve()
    if problem.status == cp.OPTIMAL:
        ub = t.value
        Sigma_opt = Sigma.value
    else:
        lb = t.value
    
print('rho_max =', t.value)
print('Sigma =', Sigma_opt)

rho_max = 0.6201171875
Sigma = [[ 4.29250754  0.38055536  1.51464781  1.56393608 -0.0283164 ]
 [ 0.38055536  2.91431241 -1.24748749  1.29813421  0.0105029 ]
 [ 1.51464781 -1.24748749  1.39150015  0.12180977  0.39674711]
 [ 1.56393608  1.29813421  0.12180977  1.50616447  0.42117708]
 [-0.0283164   0.0105029   0.39674711  0.42117708  0.32092653]]


\
\
\
\
\
\
\
\
\
\
\
\

![7.19](opt_7_19.jpg)

## Решение
Единственной проблемой, которая возникает при подстановке данных ограничений и самой функции, которую надо минимизировать, в солвер, является то, что солвер не хочет работать с органичением $1^Tx = 1$. Но мы, очевидно, можем решить задачу и не нормализуя $x$, и нормализовать его только в конце.

In [5]:
from  rank_one_nmf_data import *

In [27]:
x = cp.Variable((m,), pos=True)
y = cp.Variable((n,), pos=True)
A_var = cp.Variable((m, n), pos=True)

A_cons = [A_var[i][j] == A[i][j] for i in range(m) for j in range(n) if A[i][j]]

res = cp.vstack([x[i] * y for i in range(m)])
rel_dev = cp.maximum(cp.multiply(A_var, res ** -1), cp.multiply(A_var ** -1, res))
obj = cp.Minimize(cp.sum(rel_dev))
problem = cp.Problem(obj, A_cons)

opt_rel_dev = (problem.solve(gp=True) - m*n) / (m*n)
alpha = np.sum(x.value)
x.value /= alpha
y.value *= alpha

print('Optimal average relative deviation:', opt_rel_dev)
print('Optimal value of matrix A:\n', A_var.value)
print('Optimal value of x:', x.value)
print('Optimal values of y:', y.value)            

Optimal average relative deviation: 0.18391496985743203
Optimal value of matrix A:
 [[0.27282048 0.37342561 0.18510414 0.45223269 0.5848406  0.36126553
  0.37028095 0.34383474 0.5085739  0.06480448 0.58995068 0.22982494
  0.19755616 0.1863894  0.33082705 0.14550383 0.39270393 0.47141881
  0.46486046 0.64083187]
 [0.10841642 0.33346782 0.17520291 0.43914664 0.45853889 0.06985675
  0.06059047 0.26549556 0.39574758 0.14553939 0.46197066 0.18019214
  0.1981993  0.12177391 0.54777822 0.11408094 0.35017423 0.43545713
  0.26369495 0.57224625]
 [0.24832293 0.30963204 0.12311379 0.39262155 0.45630691 0.17322236
  0.17204251 0.2643305  0.39238157 0.09490156 0.46029395 0.17931504
  0.17351015 0.13222095 0.3315928  0.13267111 0.27389309 0.39812276
  0.30976426 0.53134914]
 [0.23350656 0.4674108  0.2237259  0.60717871 0.68882149 0.15775697
  0.23658541 0.39928063 0.56825808 0.18324137 0.66483881 0.19081269
  0.27265226 0.18126272 0.50055839 0.24847976 0.4625242  0.60693718
  0.40546329 0.80210204]
